<a href="https://colab.research.google.com/github/ModelX20/Open_Project_Winter_2025/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1 Assignmentt: Quantum Measurement Dataset Foundations

Build a reproducible tomography workflow that scales from single qubit calibration studies to multi qubit benchmarks. Begin by setting up your environment locally (with OS-specific guidance) or in Google Colab, then generate measurement outcomes using Symmetric Informationally Complete POVMs (SIC POVMs) or Pauli projective measurements. Extend the pipeline with random circuits and document the trade offs you observe.

**Task roadmap**
1. Set up and document your environment.
2. Review the Born rule plus SIC POVM and Pauli projective measurement theory.
3. Generate and visualize QST datasets.
4. Perform single qubit tomography
5. Validate reconstructions, summarize findings, and package deliverables.

> Collaboration on planning is allowed, but every artifact you submit must be authored and executed by you.

## Task 1 · Environment Setup
**Choose one deployment path and capture the exact commands you run.**

### Local virtual environment (recommended)
- **macOS / Linux:**
  1. `python3 -m venv .venv`
  2. `source .venv/bin/activate`
  3. `python -m pip install --upgrade pip wheel`
- **Windows (PowerShell):**
  1. `py -3 -m venv .venv`
  2. `.venv\Scripts\Activate.ps1`
  3. `python -m pip install --upgrade pip wheel`

### Google Colab fallback
- Create a new notebook at https://colab.research.google.com and enable a GPU if available.
- Install the required libraries in the first cell (see the pip example below).
- Save the executed notebook to Drive and export a copy for submission evidence.

### Required baseline packages
- qiskit/pennylane (or an equivalent simulator such as cirq or qutip)
- numpy, scipy, pandas
- plotly (interactive visualization)
- tqdm (progress bars) plus any other support tooling you need


In [ ]:
# Run inside your activated virtual environment or a Colab cell.
# Feel free to adjust versions based on your simulator choice.
!python -m pip install  pennylane numpy scipy pandas plotly tqdm nbformat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 122.6 MB/s eta 0:00:00


## Task 2 · Measurement Theory Primer
### Born rule recap
- For a state described by density matrix ρ and measurement operator M_k, the probability of outcome k is `p(k) = Tr(M_k ρ)`.
- For projective measurements, `M_k = P_k` with `P_k^2 = P_k` and `∑_k P_k = I`. For POVMs, `M_k = E_k` where each `E_k` is positive semi-definite and `∑_k E_k = I`.
- Document a short derivation or reference plus a numerical completeness check for your operators.

### SIC POVM vs. Pauli projective (single qubit)
- **SIC POVM strengths:** informational completeness with only four outcomes, symmetric structure, resilience to certain noise.
- **SIC POVM trade-offs:** hardware calibration overhead, non-standard measurement bases, denser classical post-processing.
- **Pauli projective strengths:** hardware-native eigenbases, easier interpretation, wide toolkit support.
- **Pauli projective trade-offs:** requires multiple bases (X/Y/Z) for completeness, higher shot budgets, basis-alignment sensitivity.

Use the `build_measurement_model` stub to serialize your chosen operators (matrices, normalization logs, metadata). Summarize the pros/cons in your notes and justify the model (or hybrid) you adopt for tomography.

## Born Rule Recap

For a quantum state described by a density matrix ρ and a measurement
operator Mₖ, the probability of outcome k is given by:

p(k) = Tr(Mₖ ρ)

### Measurement operator conditions

**Projective measurements**
- Mₖ = Pₖ
- Pₖ² = Pₖ
- ∑ₖ Pₖ = I

**POVM measurements**
- Mₖ = Eₖ
- Eₖ ≥ 0
- ∑ₖ Eₖ = I

In this task, all measurement operators are numerically checked for
positivity and completeness.


In [ ]:
import numpy as np
import json
import pathlib
from typing import Dict, Any


In [ ]:
def pauli_projective_measurements():
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])

    paulis = {"X": X, "Y": Y, "Z": Z}
    measurements = {}

    for name, P in paulis.items():
        eigvals, eigvecs = np.linalg.eigh(P)
        projectors = []
        for i in range(2):
            v = eigvecs[:, i]
            projectors.append(np.outer(v, v.conj()))
        measurements[name] = projectors

    return measurements


In [ ]:
def sic_povm_single_qubit():
    bloch_vectors = [
        np.array([ 1,  1,  1]) / np.sqrt(3),
        np.array([ 1, -1, -1]) / np.sqrt(3),
        np.array([-1,  1, -1]) / np.sqrt(3),
        np.array([-1, -1,  1]) / np.sqrt(3),
    ]

    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])
    pauli = [X, Y, Z]

    povm = []
    for r in bloch_vectors:
        rho = 0.5 * (np.eye(2) + sum(r[i] * pauli[i] for i in range(3)))
        povm.append(0.5 * rho)

    return povm


In [ ]:
def verify_measurement(measurements):
    dim = measurements[0].shape[0]
    total = np.zeros((dim, dim), dtype=complex)

    for M in measurements:
        eigvals = np.linalg.eigvalsh(M)
        assert np.all(eigvals >= -1e-10), "Operator not positive"
        total += M

    assert np.allclose(total, np.eye(dim)), "Not complete"
    return "✔ verified"


In [ ]:
pauli = pauli_projective_measurements()
for basis, ops in pauli.items():
    print(basis, verify_measurement(ops))

sic = sic_povm_single_qubit()
print("SIC POVM", verify_measurement(sic))


X ✔ verified
Y ✔ verified
Z ✔ verified
SIC POVM ✔ verified


## SIC POVM vs Pauli Projective (Single Qubit)

**SIC POVM – Strengths**
- Informational completeness with four outcomes
- Symmetric structure
- Measurement-efficient

**SIC POVM – Trade-offs**
- Calibration overhead
- Non-standard measurement bases
- More complex post-processing

**Pauli Projective – Strengths**
- Hardware-native eigenbases
- Simple interpretation
- Wide tooling support

**Pauli Projective – Trade-offs**
- Requires multiple bases (X, Y, Z)
- Higher shot budgets
- Basis-alignment sensitivity


## Measurement Model Choice

Pauli projective measurements are chosen as the primary tomography model
due to their simplicity and hardware compatibility.

SIC POVMs are included as a secondary reference model to demonstrate
informational completeness and support ML benchmarking.


### Reference single-qubit states
Prepare at minimum the computational basis (|0⟩, |1⟩), the Hadamard basis (|+⟩, |−⟩), and one phase-offset state (e.g., `( |0⟩ + i |1⟩ ) / √2`). Document how you synthesize each state in circuit form and store a textual or JSON summary of the gates used. You may optionally include mixed states by applying depolarizing or amplitude damping channels.

In [ ]:
from typing import Dict, Any
import pathlib

def build_measurement_model(config_path: pathlib.Path) -> Dict[str, Any]:
    """
    Stub for constructing or loading the measurement operators you plan to use.
    Populate the return value with operator definitions, normalization checks, and metadata.
    """
    # TODO: implement SIC POVM or Pauli projective operator assembly here.
    raise NotImplementedError("Create your measurement operator assembly here.")

In [ ]:
def build_measurement_model(config_path: pathlib.Path) -> Dict[str, Any]:
    pauli = pauli_projective_measurements()
    sic = sic_povm_single_qubit()

    model = {
        "operators": {
            "pauli_projective": {
                b: [P.tolist() for P in ops] for b, ops in pauli.items()
            },
            "sic_povm": [E.tolist() for E in sic]
        },
        "normalization_checks": {
            "pauli": {b: np.allclose(sum(ops), np.eye(2)) for b, ops in pauli.items()},
            "sic_povm": np.allclose(sum(sic), np.eye(2))
        },
        "metadata": {
            "system": "single-qubit",
            "born_rule": "p(k) = Tr(M_k rho)",
            "reference": "Renes et al., JMP 45, 2171 (2004)"
        }
    }

    with open(config_path, "w") as f:
        json.dump(model, f, indent=2)

    return model


In [ ]:
def density(psi):
    return np.outer(psi, psi.conj())

states = {
    "|0>": np.array([1, 0]),
    "|1>": np.array([0, 1]),
    "|+>": np.array([1, 1]) / np.sqrt(2),
    "|->": np.array([1, -1]) / np.sqrt(2),
    "(|0>+i|1>)/√2": np.array([1, 1j]) / np.sqrt(2),
}

density_states = {name: density(psi) for name, psi in states.items()}


In [ ]:
state_preparation_log = {
    "|0>": ["I"],
    "|1>": ["X"],
    "|+>": ["H"],
    "|->": ["X", "H"],
    "(|0>+i|1>)/√2": ["H", "S"]
}

with open("state_preparation.json", "w") as f:
    json.dump(state_preparation_log, f, indent=2)

state_preparation_log


{'|0>': ['I'],
 '|1>': ['X'],
 '|+>': ['H'],
 '|->': ['X', 'H'],
 '(|0>+i|1>)/√2': ['H', 'S']}

In [ ]:
#@title helper functions for density matrix visualization

import numpy as np
import plotly.graph_objects as go
from fractions import Fraction

_CUBE_FACES = (
    (0, 1, 2), (0, 2, 3),  # bottom
    (4, 5, 6), (4, 6, 7),  # top
    (0, 1, 5), (0, 5, 4),
    (1, 2, 6), (1, 6, 5),
    (2, 3, 7), (2, 7, 6),
    (3, 0, 4), (3, 4, 7)
 )

def _phase_to_pi_string(angle_rad: float) -> str:
    """Format a phase angle as a simplified multiple of π."""
    if np.isclose(angle_rad, 0.0):
        return "0"
    multiple = angle_rad / np.pi
    frac = Fraction(multiple).limit_denominator(16)
    numerator = frac.numerator
    denominator = frac.denominator
    sign = "-" if numerator < 0 else ""
    numerator = abs(numerator)
    if denominator == 1:
        magnitude = f"{numerator}" if numerator != 1 else ""
    else:
        magnitude = f"{numerator}/{denominator}"
    return f"{sign}{magnitude}π" if magnitude else f"{sign}π"

def plot_density_matrix_histogram(rho, basis_labels=None, title="Density matrix (|ρ_ij| as bar height, phase as color)"):
    """Render a density matrix as a grid of solid histogram bars with phase coloring."""
    rho = np.asarray(rho)
    if rho.ndim != 2 or rho.shape[0] != rho.shape[1]:
        raise ValueError("rho must be a square matrix")

    dim = rho.shape[0]
    mags = np.abs(rho)
    phases = np.angle(rho)
    x_vals = np.arange(dim)
    y_vals = np.arange(dim)

    if basis_labels is None:
        basis_labels = [str(i) for i in range(dim)]

    meshes = []
    colorbar_added = False
    for i in range(dim):
        for j in range(dim):
            height = mags[i, j]
            phase = phases[i, j]
            x0, x1 = i - 0.45, i + 0.45
            y0, y1 = j - 0.45, j + 0.45
            vertices = (
                (x0, y0, 0.0), (x1, y0, 0.0), (x1, y1, 0.0), (x0, y1, 0.0),
                (x0, y0, height), (x1, y0, height), (x1, y1, height), (x0, y1, height)
            )
            x_coords, y_coords, z_coords = zip(*vertices)
            i_idx, j_idx, k_idx = zip(*_CUBE_FACES)
            phase_pi = _phase_to_pi_string(phase)
            mesh = go.Mesh3d(
                x=x_coords,
                y=y_coords,
                z=z_coords,
                i=i_idx,
                j=j_idx,
                k=k_idx,
                intensity=[phase] * len(vertices),
                colorscale="HSV",
                cmin=-np.pi,
                cmax=np.pi,
                showscale=not colorbar_added,
                colorbar=dict(
                    title="phase ",
                    tickvals=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
                    ticktext=["-π", "-π/2", "0", "π/2", "π"]
                ) if not colorbar_added else None,
                opacity=1.0,
                flatshading=False,
                hovertemplate=
                    f"i={i}, j={j}<br>|ρ_ij|={height:.3f}<br>arg(ρ_ij)={phase_pi}<extra></extra>",
                lighting=dict(ambient=0.6, diffuse=0.7)
            )
            meshes.append(mesh)
            colorbar_added = True

    fig = go.Figure(data=meshes)
    fig.update_layout(
        scene=dict(
            xaxis=dict(
                title="i",
                tickmode="array",
                tickvals=x_vals,
                ticktext=basis_labels
            ),
            yaxis=dict(
                title="j",
                tickmode="array",
                tickvals=y_vals,
                ticktext=basis_labels
            ),
            zaxis=dict(title="|ρ_ij|"),
            aspectratio=dict(x=1, y=1, z=0.7)
        ),
        title=title,
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig.show()


In [ ]:
from IPython.display import display, clear_output
import time

labels_1q = ["0", "1"]

state_order = [
    "|0>",
    "|1>",
    "|+>",
    "|->",
    "(|0>+i|1>)/√2"
]

for name in state_order:
    print(f"Plotting state {name}")

    plot_density_matrix_histogram(
        density_states[name],
        basis_labels=labels_1q,
        title=f"Density matrix of state {name}"
    )

    time.sleep(0.5)

Plotting state |0>


Plotting state |1>


Plotting state |+>


Plotting state |->


Plotting state (|0>+i|1>)/√2


In [ ]:
A = np.random.randn(2, 2) + 1j * np.random.randn(2, 2)
rho_mixed = A @ A.conj().T
rho_mixed /= np.trace(rho_mixed)

plot_density_matrix_histogram(
    rho_mixed,
    basis_labels=["0", "1"],
    title="Example reconstructed mixed density matrix"
)


### Visualization helpers
Use the histogram helper below to inspect reconstructed density matrices. Include screenshots or exported HTML for a few representative states in your report.

In [ ]:
# Demonstration: random 2-qubit density matrix
dim = 4
A = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
rho = A @ A.conj().T
rho = rho / np.trace(rho)  # normalize

labels = ["00", "01", "10", "11"]
plot_density_matrix_histogram(rho, basis_labels=labels, title="Random 2-qubit state (density matrix)")

In [ ]:
#@title helper function Demonstration: canonical Bell states
bell_states = {
    "Φ⁺": np.array([1, 0, 0, 1], dtype=complex) / np.sqrt(2),
    "Φ⁻": np.array([1, 0, 0, -1], dtype=complex) / np.sqrt(2),
    "Ψ⁺": np.array([0, 1, 1, 0], dtype=complex) / np.sqrt(2),
    "Ψ⁻": np.array([0, 1, -1, 0], dtype=complex) / np.sqrt(2)
}

for name, state in bell_states.items():
    density_matrix = np.outer(state, state.conj())
    plot_density_matrix_histogram(
        density_matrix,
        basis_labels=["00", "01", "10", "11"],
        title=f"Bell state {name} (density matrix)"
    )

## Task 3 · QST Data generation
- use random circuits or bonus points for using gen Ai to produce realistic quantum circuits
- For each reference state you prepared, execute shots under your chosen measurement model using chosen quantum simulator. Record raw counts and computed probabilities.
- Store measurement data (`single_qubit_<state>.npx` or `.npy`)

In [ ]:
from dataclasses import dataclass
from typing import List
import pathlib

@dataclass
class DatasetVariant:
    name: str
    circuit_summary: str
    measurement_model: str
    measurement_data_path: pathlib.Path
    metadata_path: pathlib.Path
    density_matrix_path: pathlib.Path

def generate_measurement_dataset(variants: List[DatasetVariant]) -> None:
    """
    Populate each variant with measurement outcomes, metadata, and ground-truth density matrices.
    Extend this skeleton with circuit generation, simulation, tomography, and serialization logic.
    """
    # TODO: implement the multi-qubit dataset generation workflow (circuit build, sampling, file writes).
    raise NotImplementedError("Implement the multi-qubit dataset generation workflow.")

In [1]:
import numpy as np
import json
import pathlib
from dataclasses import dataclass
from typing import List


In [14]:
@dataclass
class DatasetVariant:
    name: str
    circuit_summary: str
    measurement_model: str
    measurement_data_path: pathlib.Path
    metadata_path: pathlib.Path
    density_matrix_path: pathlib.Path


In [3]:
def kron(*ops):
    out = ops[0]
    for op in ops[1:]:
        out = np.kron(out, op)
    return out


I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
H = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]])

CNOT = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])


In [4]:
ket_00 = np.array([1, 0, 0, 0], dtype=complex)

two_qubit_circuits = {
    "Phi_plus": [kron(H, I), CNOT],
    "Phi_minus": [kron(H, I), CNOT, kron(Z, I)],
    "Psi_plus": [kron(H, I), CNOT, kron(X, I)],
    "Psi_minus": [kron(H, I), CNOT, kron(X, I), kron(Z, I)],
}


In [5]:
import random

def random_single_qubit_gate():
    gates = {
        "I": I,
        "X": X,
        "H": H,
        "Z": Z
    }
    name = random.choice(list(gates.keys()))
    return name, gates[name]


def random_two_qubit_circuit(depth=4, entangle_prob=0.5):
    """
    Generates a random 2-qubit circuit as a list of unitary matrices
    and a human-readable circuit summary.
    """
    gates = []
    summary = []

    for _ in range(depth):
        g1_name, g1 = random_single_qubit_gate()
        g2_name, g2 = random_single_qubit_gate()

        gates.append(kron(g1, g2))
        summary.append(f"{g1_name}⊗{g2_name}")

        if random.random() < entangle_prob:
            gates.append(CNOT)
            summary.append("CNOT")

    return gates, " → ".join(summary)


In [6]:
num_random_states = 4  # keep small and clean

two_qubit_states = {}
two_qubit_density_states = {}
two_qubit_circuit_summaries = {}

ket_00 = np.array([1, 0, 0, 0], dtype=complex)

for i in range(num_random_states):
    gates, summary = random_two_qubit_circuit(depth=4)

    psi = ket_00
    for g in gates:
        psi = g @ psi

    state_name = f"random_state_{i}"

    two_qubit_states[state_name] = psi
    two_qubit_density_states[state_name] = np.outer(psi, psi.conj())
    two_qubit_circuit_summaries[state_name] = summary


In [7]:
def two_qubit_pauli_projectors():
    paulis = {"X": X, "Z": Z}
    projectors = {}

    for a in paulis:
        for b in paulis:
            basis = f"{a}{b}"
            _, ea = np.linalg.eigh(paulis[a])
            _, eb = np.linalg.eigh(paulis[b])

            ops = []
            for i in range(2):
                for j in range(2):
                    ops.append(
                        kron(
                            np.outer(ea[:, i], ea[:, i].conj()),
                            np.outer(eb[:, j], eb[:, j].conj())
                        )
                    )
            projectors[basis] = ops

    return projectors


two_qubit_measurements = two_qubit_pauli_projectors()


In [8]:
def simulate_two_qubit_measurements(rho, projectors, shots=2000):
    results = {}

    for basis, ops in projectors.items():
        probs = np.array([np.real(np.trace(op @ rho)) for op in ops])
        probs = np.maximum(probs, 0)
        probs /= np.sum(probs)

        counts = np.random.multinomial(shots, probs)

        results[basis] = {
            "counts": counts.tolist(),
            "probabilities": probs.tolist()
        }

    return results


In [9]:
def generate_measurement_dataset(
    variants: List[DatasetVariant],
    shots: int = 2000
) -> None:

    for variant in variants:
        rho = two_qubit_density_states[variant.name]

        measurement_data = simulate_two_qubit_measurements(
            rho,
            two_qubit_measurements,
            shots=shots
        )

        np.save(variant.measurement_data_path, measurement_data, allow_pickle=True)
        np.save(variant.density_matrix_path, rho)

        metadata = {
            "state": variant.name,
            "circuit_summary": variant.circuit_summary,
            "measurement_model": variant.measurement_model,
            "shots": shots
        }

        with open(variant.metadata_path, "w") as f:
            json.dump(metadata, f, indent=2)


In [10]:
base_dir = pathlib.Path("qst_dataset_2qubit")
base_dir.mkdir(exist_ok=True)

variants = []

for state in two_qubit_states.keys():
    variants.append(
        DatasetVariant(
            name=state,
            circuit_summary=two_qubit_circuit_summaries[state],
            measurement_model="Two-qubit Pauli (XX, XZ, ZX, ZZ)",
            measurement_data_path=base_dir / f"two_qubit_{state}.npy",
            metadata_path=base_dir / f"two_qubit_{state}_meta.json",
            density_matrix_path=base_dir / f"two_qubit_{state}_rho.npy",
        )
    )

In [11]:
generate_measurement_dataset(variants, shots=2000)
print("Task 3 (two-qubit) dataset generation complete.")


Task 3 (two-qubit) dataset generation complete.


In [17]:
sample = np.load(
    base_dir / "two_qubit_random_state_0.npy",
    allow_pickle=True
).item()

sample


{'XX': {'counts': [994, 1006, 0, 0], 'probabilities': [0.5, 0.5, 0.0, 0.0]},
 'XZ': {'counts': [0, 2000, 0, 0], 'probabilities': [0.0, 1.0, 0.0, 0.0]},
 'ZX': {'counts': [480, 521, 516, 483],
  'probabilities': [0.25, 0.25, 0.25, 0.25]},
 'ZZ': {'counts': [0, 988, 0, 1012], 'probabilities': [0.0, 0.5, 0.0, 0.5]}}

In [18]:
import json

with open(base_dir / "two_qubit_random_state_0_meta.json") as f:
    meta = json.load(f)

meta


{'state': 'random_state_0',
 'circuit_summary': 'X⊗I → CNOT → Z⊗I → H⊗I → CNOT → I⊗X → CNOT',
 'measurement_model': 'Two-qubit Pauli (XX, XZ, ZX, ZZ)',
 'shots': 2000}

## Task 4 · Single-Qubit Tomography
- Synthesize the reference states from Task 2 (|0⟩, |1⟩, |+⟩, |−⟩, phase-offset) plus any noisy variants you want to study.
- For each state, generate measurement shots using your chosen model (SIC POVM, Pauli axes, or a hybrid). Capture raw counts, probabilities, and seeds.
- Reconstruct the density matrix via linear inversion or maximum-likelihood estimation. Compare results across measurement models when possible.
- Quantify reconstruction fidelity (e.g., fidelity, trace distance, Bloch vector error) and tabulate the metrics.
- save data under `data/single_qubit/`: measurement outcomes (`.npx`/`.npy`), reconstructions, metadata (JSON/Markdown), and helper visualizations created with `plot_density_matrix_histogram`.

In [ ]:
#your code ..

## Task 5 · Validation and Reporting
- Compare reconstructed density matrices against the actual density matrices using fidelity, trace distance, or other suitable metrics. Plot trends (per circuit depth, shot count, or measurement model).
- Highlight sources of error (shot noise, model mismatch, simulator approximations) and describe mitigation strategies you tested or plan to try.
- Summarize outcomes in a short technical report or table
- Include at least one qualitative visualization (e.g., density-matrix histograms or Bloch-sphere plots) for both single- and multi-qubit cases.
- Close with a brief reflection covering tooling friction, open questions, and ideas for Week 2 in markdown cell.

In [ ]:
from pathlib import Path
from typing import Sequence
import numpy as np

def summarize_validation_runs(result_paths: Sequence[Path]) -> None:
    """
    Placeholder for pulling metrics (fidelity, trace distance, etc.) from stored validation artifacts.
    Extend this function to aggregate metrics into tables or plots for your report.
    """
    # TODO: load metrics, compute aggregates, and emit summaries/plots.
    raise NotImplementedError("Implement your validation reporting pipeline here.")

## Submission Checklist
- Environment setup: env directory (requirements.txt or environment.yml), OS diagnostics, and import verification logs/notebook cells.
- Measurement theory notes: Born rule recap, SIC POVM vs. Pauli analysis, operator definitions, and validation checks.
- Data artifacts: `.npx`/`.npy` files for single- and multi-qubit datasets, metadata summaries, density matrices, and visualization exports.
- Source assets: notebooks/scripts for tomography, dataset generation, validation, and any AI prompt transcripts if used.
- Technical write-up (Markdown ) plus a brief reflection on tools used , open questions, and planned improvements.

-----